<a href="https://colab.research.google.com/github/IIF0403/Project/blob/master/FCN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%matplotlib inline
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import files
from google.colab import output

In [10]:
seed1 = 14
seed2 = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Function to load UCR time_series data
def load_data(Dataset):
  url_raw = 'https://raw.githubusercontent.com/IIF0403/timeseries/master/data/'
  url_train = url_raw + Dataset+'/'+Dataset+'_TRAIN'
  url_test = url_raw + Dataset+'/'+Dataset+'_TEST'

  data_train = pd.read_csv(url_train,header=None)
  data_test = pd.read_csv(url_test, header=None)

  data = pd.concat((data_train, data_test))

  #Want all datasets to have classes starting from 0
  Y = data.values[:,0]
  classes = len(np.unique(Y))
  Y_transformed = ( (Y-Y.min())/(Y.max()-Y.min()) )*(classes-1)
  data[data.columns[0]] = Y_transformed

  return data

def explore_data(data):
  #data = load_data(Dataset)

  time_series = data.shape[0]
  classes = len( np.unique(data.iloc[:,0]))
  T = data.shape[1]-1

  #print("#: ", time_series)
  #print("C: ", classes)
  #print("T: ", T)

  return time_series, classes, T

#Function prepares data and removes labels from y_train
def prepare_data(data, test_data_size=0.2, unlabeled_data_size = 0.9 ):
  data = data.sample(frac = 1)  #shuffle the data

  #Split data into 80% train data and 20% test data
  data_train_before, data_test = train_test_split(data, test_size=test_data_size, random_state=seed1)
  
  #Split the train data into 90% unlabeled data and 10% labeled data
  data_train = data_train_before
  while (  len(np.unique(data_train.iloc[:,0])) != (classes+1)  ): #Make sure that all labels are included in the labeled train data
    data_train_before = data_train_before.sample(frac=1) #Shuffle the original train data
    train_labeled, train_unlabeled = train_test_split(data_train_before, test_size=unlabeled_data_size, random_state=seed2)  
    train_unlabeled[train_unlabeled.columns[0]] = -1 #Set labels to -1
    train = pd.concat((train_labeled,train_unlabeled))
    data_train = train.sample(frac = 1)  #shuffle the train data
    #print("np unique:", len(np.unique(data_train.iloc[:,0])))


  x_train = data_train.iloc[:,1:].to_numpy()
  y_train = data_train.iloc[:,0].to_numpy()
  x_test = data_test.iloc[:,1:].to_numpy()
  y_test = data_test.iloc[:,0].to_numpy() 

  x_train=x_train[:,np.newaxis,:] 
  x_test=x_test[:,np.newaxis,:]


  #Torch
  x_train = torch.from_numpy(x_train).to(device)
  y_train = torch.from_numpy(y_train).to(device)
  x_test = torch.from_numpy(x_test).to(device)
  y_test = torch.from_numpy(y_test).to(device)

  return x_train, y_train, x_test, y_test



In [11]:
#FCN model with only classification
class class_model(nn.Module):
  def __init__(self, classes):
    super(class_model, self).__init__()
    self.conv1 = nn.Conv1d(1, 128, 9, padding=(9 // 2))
    self.bnorm1 = nn.BatchNorm1d(128)        
    self.conv2 = nn.Conv1d(128, 256, 5, padding=(5 // 2))
    self.bnorm2 = nn.BatchNorm1d(256)
    self.conv3 = nn.Conv1d(256, 128, 3, padding=(3 // 2))
    self.bnorm3 = nn.BatchNorm1d(128)        
    self.classification_head = nn.Linear(128, classes)

  def forward(self, x_class):
    b1_class = F.relu(self.bnorm1(self.conv1(x_class)))
    b2_class = F.relu(self.bnorm2(self.conv2(b1_class)))
    b3_class = F.relu(self.bnorm3(self.conv3(b2_class)))

    features_class = torch.mean(b3_class, 2)   
    out_class = self.classification_head(features_class)

    return out_class


In [12]:
#Some necessary functions

#Function to shuffle data
def shuffle(X,Y):
  index = np.array( [int(i) for i in range(X.shape[0]) ] )
  np.random.shuffle(index)
  return X[index], Y[index]

#Function to export resulting data in Excel file
def to_Excel(Results, output_name):
  columns = ["Dataset","dataload","Model","Epochs", "alpha", "stride", "horizon", "batch size", "Accuracy", "Classification loss", "Forecasting loss"]
  dataframe = pd.DataFrame(Results, columns = columns)
  # create excel writer object
  writer = pd.ExcelWriter(output_name)
  # write dataframe to excel
  dataframe.to_excel(writer)
  # save the excel
  writer.save()
  files.download(output_name)
  print('DataFrame is written successfully to Excel File.') 

#Function to play sound when code is finished
def sound():
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/9/91/Sound4.wav").play()')



In [23]:
dataload = 1
Datasets = ["Coffee", "CBF", "ECG200", "FaceFour","ItalyPowerDemand","Lighting7", "OliveOil"]
Dataset = Datasets[1]

Data = load_data(Dataset)
time_series, classes, T = explore_data(Data)
x_train, y_train, x_test, y_test = prepare_data(Data)

print("x_train shape: ", x_train.shape, "  y_train shape: ",x_train.shape)
print("x_test shape: ", x_test.shape, "  y_test shape: ",x_test.shape)




x_train shape:  torch.Size([744, 1, 128])   y_train shape:  torch.Size([744, 1, 128])
x_test shape:  torch.Size([186, 1, 128])   y_test shape:  torch.Size([186, 1, 128])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:

batch_size = 34
max_epochs = 100

class_net = class_model(classes).to(device)

optimizer_classification = torch.optim.Adam(class_net.parameters(), lr=1e-4)
loss_func_classification = nn.CrossEntropyLoss()

accuracies_classification = []
losses_classification = []

x_train_labeled = x_train[y_train!=-1] #the x part of the training set that has a label
y_train_labeled = y_train[y_train!=-1] #the y part of the training set that has a label

N_L = x_train_labeled.shape[0]

print(N_L)

print("Dataset: ", Dataset)
for e in range(max_epochs):
  x_train, y_train = shuffle(x_train,y_train) #Shuffle training data

  x_train_labeled = x_train[y_train!=-1] #the x part of the training set that has a label
  y_train_labeled = y_train[y_train!=-1] #the y part of the training set that has a label

  for i in range(0, N_L, batch_size):
    if ( i+batch_size <= x_train_labeled.shape[0]):
      x_batch = x_train_labeled[i:i+batch_size]
      y_batch = y_train_labeled[i:i+batch_size]
    else:
      x_batch = x_train_labeled[i:]
      y_batch = y_train_labeled[i:]

    y_hat = class_net(x_train_labeled.float())

    loss_cl = loss_func_classification(y_hat, y_train_labeled.long())

    optimizer_classification.zero_grad()
    loss_cl.backward()
    optimizer_classification.step()

    loss_classification = loss_cl.item()
    losses_classification.append(loss_classification)

  y_hat_classsification = class_net(x_test.float())
  predicted_classification = torch.max(y_hat_classsification,1)[1]

  accuracy_classification = accuracy_score(y_test, predicted_classification) 
  accuracies_classification.append(accuracy_classification)

  avg_loss_classification = np.mean(losses_classification)

  print("----------------------")
  print("E: ", e, " Acc: ", accuracy_classification, "bs: ", batch_size)

  if accuracy_classification==1.0:
    break;

result = [Dataset, dataload, "Classification", e+1, -1, -1, -1, batch_size, max(accuracies_classification),avg_loss_classification, -1, lr]
print(result)
